In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
from skimage import io, transform
import PIL
import numpy as np
import matplotlib.pyplot as plt
import os

In [25]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./image_files', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [26]:
def imshow(img):
    unnormalized_image = img / 2 + 0.5     # unnormalize
    npimg = unnormalized_image.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    

In [30]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.fc1 = nn.Linear(32 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 32 * 2 * 2)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [31]:
net = Net().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

In [32]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs_cpu, labels_cpu = data
        inputs, labels = inputs_cpu.cuda(), labels_cpu.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0


[1,   100] loss: 2.301
[1,   200] loss: 2.286
[1,   300] loss: 2.206
[1,   400] loss: 2.072
[1,   500] loss: 1.959
[1,   600] loss: 1.912
[1,   700] loss: 1.841
[1,   800] loss: 1.792
[1,   900] loss: 1.750
[1,  1000] loss: 1.734
[1,  1100] loss: 1.676
[1,  1200] loss: 1.670
[1,  1300] loss: 1.594
[1,  1400] loss: 1.604
[1,  1500] loss: 1.555
[1,  1600] loss: 1.525
[1,  1700] loss: 1.522
[1,  1800] loss: 1.533
[1,  1900] loss: 1.498
[1,  2000] loss: 1.431
[1,  2100] loss: 1.461
[1,  2200] loss: 1.400
[1,  2300] loss: 1.467
[1,  2400] loss: 1.390
[1,  2500] loss: 1.414
[1,  2600] loss: 1.382
[1,  2700] loss: 1.350
[1,  2800] loss: 1.365
[1,  2900] loss: 1.359
[1,  3000] loss: 1.361
[1,  3100] loss: 1.373
[2,   100] loss: 1.314
[2,   200] loss: 1.322
[2,   300] loss: 1.307
[2,   400] loss: 1.275
[2,   500] loss: 1.294
[2,   600] loss: 1.230
[2,   700] loss: 1.267
[2,   800] loss: 1.286
[2,   900] loss: 1.238
[2,  1000] loss: 1.247
[2,  1100] loss: 1.309
[2,  1200] loss: 1.287
[2,  1300] 

In [34]:
testset = torchvision.datasets.CIFAR10(root='./image_files', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)


In [35]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images_cpu, labels_cpu = data
        images, labels = images_cpu.cuda(), labels_cpu.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 65 %


In [37]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images_cpu, labels_cpu = data
        images, labels = images_cpu.cuda(), labels_cpu.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of plane : 65 %
Accuracy of   car : 96 %
Accuracy of  bird : 54 %
Accuracy of   cat : 26 %
Accuracy of  deer : 55 %
Accuracy of   dog : 48 %
Accuracy of  frog : 75 %
Accuracy of horse : 56 %
Accuracy of  ship : 75 %
Accuracy of truck : 69 %
